# Lab 03 - Visualizing Geospatial Data in Plotly

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly.express as px

from pathlib import Path

%pylab inline

In [ ]:
## Only run this on Google Colab
# !pip install geopandas plotly

In [ ]:
## Only do this if you're working on Google Colab

# from google.colab import drive
# drive.mount('/gdrive')
# %cd /gdrive

In [ ]:
# when using Google Colab
# dataset_folder = Path('/gdrive/MyDrive/datasets')

# when using local folder
dataset_folder = Path('datasets/')

## Reading in the Transformed Data

To make things easier for us for the Dash translation, let's test first the visuals we want to build in a notebook.

In [ ]:
educ_sites = gpd.read_file('datasets/hotosm_phl_education_facilities.shp')
amenity_gdf = gpd.read_file('datasets/ph_educ_by_amenity.geojson', driver='GeoJSON')
operatorty_gdf = gpd.read_file('datasets/ph_educ_by_operatorty.geojson', driver='GeoJSON')

### Use the Mapbox Token

Use the default public token available to you when you sign up for an account in [Mapbox](https://mapbox.com)

Save it in a plain text file called `.mapbox_token`

We would need this token for the base map themes that are available from Mapbox.

In [ ]:
px.set_mapbox_access_token(open(".mapbox_token").read())

In [ ]:
educ_sites.head()

## Scatter Map

We previously created the scatter map using the GADM base polygons and the `educ_sites` data. 

Since we're using Mapbox, we can now just visualize the `educ_sites` directly because Mapbox handles the base map for us (with labels!)

Reference: [Plotly Documentation](https://plotly.com/python/scattermapbox/)

In [ ]:
fig = px.scatter_mapbox(educ_sites, 
                        lat=educ_sites.geometry.y,
                        lon=educ_sites.geometry.x,
                        hover_name="name",
                        color="amenity",
                        zoom=7)
fig.show()

### Cluster Map

It is also possible to show clustered numbers instead of individual points. 

But tooltips will be limited and the points will only display when the map is zoomed in.

In [ ]:
fig = px.scatter_mapbox(educ_sites, 
                        lat=educ_sites.geometry.y,
                        lon=educ_sites.geometry.x,
                        hover_name="name",
                        color="amenity",
                        zoom=7)
fig.update_traces(cluster=dict(enabled=True))
fig.show()

## Choropleth Maps

We also created choropleth maps using the pivot tables we created.

Reference: [Plotly Documentation](https://plotly.com/python/mapbox-county-choropleth/)

In the documentation, when [using GeoPandas DataFrames](https://plotly.com/python/mapbox-county-choropleth/#using-geopandas-data-frames), the geometry is accessed and there is a `locations` attribute which requires the `index` of the DataFrame.

```python
import plotly.express as px
import geopandas as gpd

df = px.data.election()
geo_df = gpd.GeoDataFrame.from_features(
    px.data.election_geojson()["features"]
).merge(df, on="district").set_index("district")

fig = px.choropleth_mapbox(geo_df,
                           geojson=geo_df.geometry,
                           locations=geo_df.index,
                           color="Joly",
                           center={"lat": 45.5517, "lon": -73.7073},
                           mapbox_style="open-street-map",
                           zoom=8.5)
fig.show()
```

A trick to show the name of the location as the index is to set is as the index of the GeoDataFrame to be used in the `plotly.express` graph.

In [ ]:
amenity_gdf_indexed = amenity_gdf.set_index('province')

In [ ]:
amenity_gdf_indexed.head()

In [ ]:
fig = px.choropleth_mapbox(amenity_gdf_indexed,
                           geojson=amenity_gdf_indexed.geometry,
                           locations=amenity_gdf_indexed.index,
                           color='school',
                           center={'lat': 12.099568, 'lon': 122.733168},
                           zoom=4)
fig.show()

In [ ]:
operatorty_gdf_indexed = operatorty_gdf.set_index('province')

In [ ]:
operatorty_gdf_indexed.head()

In [ ]:
fig = px.choropleth_mapbox(operatorty_gdf,
                           geojson=operatorty_gdf.geometry,
                           locations=operatorty_gdf.index,
                           color='public',
                           center={'lat': 12.099568, 'lon': 122.733168},
                           zoom=4)
fig.show()

### Retrieving the options for the dropdown

In our case, the DataFrame does not have the options in a single column. But it can be derived from the columns (since they are in sequence!)

We'll maximize the use of array slicing (a key Python / data wrangling skill!)

In [ ]:
amenity_gdf.columns

In [ ]:
amenity_gdf.columns[4:-1]

In [ ]:
operatorty_gdf.columns[4:-1]

## Additional visualizations

Choropleth maps are nice in showing the distribution, but sometimes the data is hard to decipher. A good way to supplement the data is by showing a bar graph of the counts on the side.

For more colormaps available, check the documentation about [colors in Plotly](https://plotly.com/python/colorscales/).

In [ ]:
sorted_df = operatorty_gdf_indexed.sort_values(by='public')
px.bar(sorted_df, x='public', y=sorted_df.index, color_continuous_scale='teal', color='public', template='plotly_white')